## 10 Year Horizon (2030)

Goal: The goal of this program is to determine which nodes(locations) and what proportion of each will satisfy the desired planning outcome (load demand, reliability, land capacity, etc...)

In [ ]:
from pyomo.environ import *
import pandas as pd
import numpy as np

m = ConcreteModel()

## Data

df_solar = pd.read_csv('Solar Supply Data Final')
df_wind = pd.read_csv('Wind Supply Data Final')
sl = list(df_solar['location']) # Solar latitude/longitude set
wl = list(df_wind['location']) # Wind latitude/longitude set
ss = list(df_solar['capacity_mw']) # Solar supply generation set
ws = list(df_wind['capacity_mw']) # Wind supply generation set
scf = list(df_solar['capacity_factor']) # Solar capacity factor set
wcf = list(df_solar['capacity_factor']) # Wind capacity factor set

## Calcuations - Demand 

expected = 2354.6 # (MW) Energy demand from EIA reference population/economic growth
retirement = 9158.4  # (MW) rate of retirement may be needed for other horizons
demand = expected + retirement # Cummilative load demand for the first 10 years

## Average Time Horizon Costs

sc = 2150700 #Average cost($) of solar per MW capacity for first 10 years
wc = 3190800 #Average cost($) of wind per MW capacity for first 10 years
ngct = 2173124 #Average cost($) of natural gas combution turbine per MW capacity for first 10 years
ngcc = 2235928 #Average cost($) of natural gas combined cycle per MW capacity for first 10 years
ngccs = 5253428 #Average cost($) of natural gas combined cycle with carbon capture per MW capacity for first 10 years

## Dictionary of Data

solar = dict(map(lambda x,y: (x,y),sl,ss)) # Dictionary of location and respective solar genreative capacities
wind = dict(map(lambda x,y: (x,y),wl,ws)) # Dictionary of locatiotn and respective wind geenrative capacities
solar_cap = dict(map(lambda x,y: (x,y),sl,scf)) # Dictionary of locatiotn and respective solar capacity factors
wind_cap = dict(map(lambda x,y: (x,y),wl,wcf)) # Dictionary of location and repsective wind capaicty factors

#### Set Decleration

In [7]:
'''Annotation: Initially configured what are now parameters as sets.  This method would have formated the results
such that respective capacities are show.  For the sake of viewing selected locations this method was revised to 
configure the following as parameters.'''

#m.ss = Set(initialize = ss, ordered = True, doc = 'Solar supply generation set')

#m.ws = Set(initialize = ws, ordered = True, doc = 'Wind supply generation set')

#m.scf = Set(initialize = scf, ordered = True, doc = 'Soalr capacity factor set')

#m.wcf = Set(initialize = wcf, ordered = True, doc = 'Wind capacity factor set')

In [64]:
m.sl = Set(initialize = sl, ordered = True, doc = 'Solar loaction set in longitude and latitude')

m.wl = Set(initialize = wl, ordered = True, doc = 'Wind location set in longitude and latitude')

#### Parameter Decleration

In [65]:
m.ss = Param(m.sl, initialize = solar, doc = 'Solar supply generation')

m.ws = Param(m.wl, initialize = wind, doc = 'Wind supply generation')

m.scf = Param(m.sl, initialize = solar_cap, doc = 'Solar capacity factors')

m.wcf = Param(m.wl, initialize = wind_cap, doc = 'Wind capacity factors')

m.sc = Param(initialize = sc, doc = 'Average solar cost per generative capacity')

m.wc = Param(initialize = wc, doc = 'Average wind cost per generative capacity')

m.ngct = Param(initialize = ngct, doc = 'Average natural gas combustion turbine cost per generative capacity')

m.ngcc = Param(initialize = ngcc, doc = 'Average natural gas combined cycle cost per generative capacity')

m.ngccs = Param(initialize = ngccs, doc = 'Average natural gas combined cyle with carbon capture cost per generative capacity')

m.D10 = Param(initialize = demand, doc = 'Expected demand for 10 year Horizon')

#### Variable Decleration

In [66]:
## Generative Capacities

'''Goal: To determine the selection of loactions and the proporiton of each selection that is sufficient to fufill the
imosed demand'''

m.S = Var(m.sl, bounds = (0.0,1.0), doc = 'Selection of solar generative capacity selected')

m.W = Var(m.wl, bounds = (0.0,1.0), doc = 'Location of wind generative capacity selected')

m.NGCT = Var(domain = PositiveReals, doc = 'Amount of NG combustion turbine generative capacity selected')

m.NGCC = Var(domain = PositiveReals, doc = 'Amount of NG combined cycle generative capacity selected')

m.NGCCS = Var(domain = PositiveReals, doc = 'Amount of NG combined cycle with carbon capture generative capacity selected')

#### Constraints

##### Demand Constraint:
This constraint is incoorporative of the amounts of generative capaicyt selected for solar, wind, and the various natural gas technolgies that meet the load demand for the planning horizon

In [67]:
## Demand Requirement Constraint
def Demand_rule(m):
    return (sum(m.S[i] for i in m.ss) + sum(m.W[i] for i in m.ws) + m.NGCT + m.NGCC + m.NGCCS) >= m.D10
m.Demand = Constraint(rule = Demand_rule)

##### Reliability Constraint:
This constraint will incorporate the capacity factor of each assessed node to the associated capacity.  This can be done via two avenues

1.)Forecasting consumption needed to be met ((capacity factor * capaciy) >= consumption)

2.)Two, proiding a capcity factor thershold to be sustained for improved system reliability.  

The first avenue may have little effect on the overall system selection.  The second avenue will have a large effect on the system.  The selection of genearative technoiges (i.e. wind and solar) will be favored as to increase reliability.  Thus as of now this is left incomplete.

In [ ]:
## Reliability Constraint
#def Capfactor_rule(m):
    #return ())
#m.Capfactor = Constraint(rule = Capfactor_rule)

##### Land Capacity Constraint:
Land Capacity Constraint: The sum of all availble land for each location for solar and wind is 
within the land mass of Indiana.  However the feasibility of how large a farm can be before ineffeciencies
are significant must also be considered.  That is the purpose of this constratin.  However with some research 
already conducted, it is yet to be determined what the optimal land occupancy for a solar or wind farm is.
Thus as of now it is left incomplete.

In [ ]:
## Land Capacity Constraint  
##def LandCap_rule(m):
    #return()
#m.LandCap = Constraint(rule = LandCap_rule)

##### Carbon Penalty Constraint:
For the purpose of promoting the use of renewables and leveraging the use of natural gas, this constraint would invoke a cost penalty after either

1.) A certain threshold of natural gas capacity is selected and a finiancial penalty is inacted

2.) A ceratain threshold of emission is propsed upon which crossed natural gas capacites are no longer considered

In all cases the affected decision variables are natural gas combustion turbine and natural gas combined cycle.  Natural gas combined cycle with carbon sequestration would not be affected, however its cost of implementation with current technolgies is so high that it's selectivity is always low.

In [ ]:
## Carbon Penalty Constraint
#def Carbon_rule(m):
    #return()
#m.Carbon = Constraint(rule = Carbon_rule)

#### Objective Function:
The goal of the objective function is to minimize the cost of the seleted generative capacities.  The goal is to treat all solar and wind locations as nodes.  To optimize the system the goal is to select the nodes (locations of solar and wind) that along with the given constraints satisy the objective.  Each location, or node, is distributed in ordered sets, distinguished by solar and wind.  Each node osnts of a generative capacity, capacity factor, etc....  The goal of this program is to determine which nodes and what proportion of each will satisfy the desired planning outcome

In [ ]:
## Cost Minimization

obj = (m.sc * sum(m.S[i] for i in m.ss) + m.wc * sum(m.W[i] for i in m.ws) + (m.ngct * m.NGCT) +  (m.ngcc * m.NGCC) + 
    (m.ngccs * m.NGCCS) )
    
m.obj = Objective(sense = minimize, expr = obj)

#### Solution

In [ ]:
## Declaritive Solution

def pyomo_postprocess(options = None, instance = None, results = None):
    m.S.display()
    m.W.display()
    m.Demand.display()
    m.obj.display()
    m.NGCT.display()
    m.NGCC.display()
    m.NGCCS.display()
    
    
if __name__=='__main__':
    from pyomo.opt import SolverFactory
    import pyomo.environ
    opt = SolverFactory('glpk')
    results = opt.solve(m)
    results.write()
    print('\nDisplaying Solution\n')
    pyomo_postprocess(None, m, results)

In [ ]:
## Detailed Solution
import pyomo.opt as po
solver = po.SolverFactory('glpk') # GNU Linear Programming Kit
result = solver.solve(m, tee=True) #tee displays realtime solver log